# Scene Boundary Paritioning
Movies are comprised of self-contained units called scenes. Scenes have a beginning and end, usually comprised of a single conversation. They most often take place in one location with a fixed number of characters. By identifying scenes in a movie, we can then begin to analyze them individually, most notably by treating a scenes's dialogue as a freestanding, indepdent conversation.

To start, we'll just be identifying two-character dialogue scenes. These are the most basic building-blocks of films: just two characters speaking together with no distractions, purely advancing the plot with their dialogue. In modern filmmaking, these scenes are usually shot in a specific manner. We can take advantage of this by looking for specific patterns of shots, to identify a few two-character dialogue scenes.

In [1]:
import sys
sys.path.append('../data_serialization')
from serialization_preprocessing_io import *
from time_reference_io import *
from scene_identification_io import *

We have saved pickle objects of various dataframes. We'll load into memory the five dataframes, but we're most interested in the two which deal with onscreen images. The each have one row per frame (screencap), with one frame per second — so each row represents one second of onscreen action.
- vision_df - contains general computer vision information on each frame, including clusterings of similar frames into "shots"
- face_df - contains information related to faces found, including their vectorized encodings, and clusters of these encodings

In [ ]:
film = 'lost_in_translation_2003'
srt_df, subtitle_df, sentence_df, vision_df, face_df = read_pickle(film)

## The A/B/A/B pattern
In modern film, two-character dialogue scenes follow a very distinct pattern. Character A speaks, then Character B, then back to A, then to B, etc. We cut back and forth between the two characters.

We look for these two Anchor shots, which are the shots of the two characters and form the A/B/A/B pattern. We can also identify Cutaway shots, which aren't part of the A/B/A/B pattern but are still part of the scene.